In [12]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import time as tm
import numpy as np

# Compañias disponibles

In [2]:
companys=pd.read_excel('compañias_bmv.xlsx')
symbols=list(companys['Symbol'])
names=list(companys['Company'])
names=[name[:-1] if  name.endswith('.') else name for name in names ]
# sorted(names)

In [42]:
companys[100:120]

,Symbol,Company
100,CHWY.MX,"Chewy, Inc."
101,LFCN.MX,China Life Insurance Company Limited
102,SNPN.MX,China Petroleum & Chemical Corporation
103,CMG.MX,"Chipotle Mexican Grill, Inc."
104,CHRD.MX,Chord Energy Corporation
105,CNK.MX,"Cinemark Holdings, Inc."
106,CSCO.MX,"Cisco Systems, Inc."
107,C.MX,Citigroup Inc.
108,CLF.MX,Cleveland-Cliffs Inc.
109,NET.MX,"Cloudflare, Inc."


# Trayendo la información de  yahoo finance

In [47]:
def get_info_compamys(*ncompanys,**kwargs):
    nsymbs=[symbols[names.index(com)]  if com not in symbols  else com for com in ncompanys]
    today=tm.localtime()
    today='-'.join([str(i) for i in list(today)[:3]])
    last_year=str(int(today[:4])-1)+today[4:]
    data = yf.download(tickers=sorted(nsymbs), start=last_year, end=today)
    def rendimiento(comp):return [comp[i]/comp[i-1]-1 for i in range(1,len(comp))]
    symbs=list(data.Close)
    cols=[names[symbols.index(sym)] for sym in symbs ]
    dates=list(data.index[1:])
    df_rend=pd.DataFrame([rendimiento(list(data.Close[c])) for c in symbs]).T
    df_rend.columns=cols
    df_rend.columns.name="Empresa"
    df_rend.index=dates
    df_rend.index.name="Date"
    monto=kwargs['monto_invertir']
    periodicidad=kwargs['periodicidad']
    beta=kwargs['beta']
    return df_rend[::-1],data,cols

In [83]:
Datos=get_info_compamys( 'Grupo Bimbo, S.A.B. de C.V',
                       'Grupo Televisa, S.A.B',
                        'Coca-Cola FEMSA, S.A.B. de C.V',
                        'Kimberly-Clark de México, S. A. B. de C. V',
                        'Grupo Carso, S.A.B. de C.V', 'CEMEX, S.A.B. de C.V',
                        monto_invertir=100,periodicidad='mensual',beta='conservador')
Datos[0]

[*********************100%***********************]  6 of 6 completed


Empresa,"Grupo Bimbo, S.A.B. de C.V","CEMEX, S.A.B. de C.V","Grupo Carso, S.A.B. de C.V","Kimberly-Clark de México, S. A. B. de C. V","Coca-Cola FEMSA, S.A.B. de C.V","Grupo Televisa, S.A.B"
Date,,,,,,
2022-09-06,0.005675,-0.010376,-0.004895,-0.001794,0.046463,-0.017682
2022-09-05,-0.020703,-0.011538,0.027515,-0.014498,-0.012433,0.004341
2022-09-02,0.024192,0.018277,-0.007925,0.031364,-0.005369,0.011577
2022-09-01,0.061160,0.025435,-0.017164,0.004764,-0.002111,-0.013391
2022-08-31,-0.046783,-0.031128,-0.046355,-0.013733,-0.012427,-0.028320
...,...,...,...,...,...,...
2021-09-14,0.077133,0.002628,0.034221,0.023431,0.017609,-0.003878
2021-09-13,0.020473,-0.008469,-0.012489,0.016466,0.000443,0.002250
2021-09-10,0.004593,0.004581,0.008296,0.008899,0.000886,-0.028611


# Graficas

#### Todas juntas

In [111]:
def graph_compamys(df,columnas=Datos[2]):
    close=df.Close
    cols=[c.strip().replace(",","").replace("S.A.B","")
          .replace("de C.V","").replace(".","").replace("S A B de C V","").replace("Inc","").strip()
          for c in columnas ]
    close.columns=cols
    close.columns.name="Empresa"
    fig = px.area(close, facet_col="Empresa", facet_col_wrap=2)
    fig.show()

In [110]:
graph_compamys(Datos[1])

#### Solo las que se seleccionan

In [107]:
def grapich(df,companys,columnas=Datos[2]):
    close=df.Close
    cols=[c.strip().replace(",","").replace("S.A.B","")
          .replace("de C.V","").replace(".","").replace("S A B de C V","").replace("Inc","").strip()
          for c in columnas ]
    close.columns=cols
    for c in companys:
        fig = px.line(close, x=df.index, y=c,title="Precio")
        fig.show()

In [108]:
grapich(Datos[1],['Grupo Televisa','Coca-Cola FEMSA','CEMEX'])

#### Velas chinas

In [140]:
symbs=list(Datos[1].Close)
today=tm.localtime()
today='-'.join([str(i) for i in list(today)[:3]])
last_year=str(int(today[:4])-1)+today[4:]
names=[c.strip().replace(",","").replace("S.A.B","")
          .replace("de C.V","").replace(".","").replace("S A B de C V","").replace("Inc","").strip()
          for c in Datos[2] ]
for sym in symbs:
    df = yf.download(sym,start=last_year, end=today)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df.Open,
                    high=df.High,
                    low=df.Low,
                    close=df.Close)])
    fig.update_layout(title='Comportamiento en el ultimo año',yaxis_title=name)
    fig.show()

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


# Ultimo precio

In [95]:
def last_price(lista_tickers):
    today=tm.localtime()
    if today.tm_mon==1:
        emp=str(today.tm_year-1)+"-12-30"
    else:
        emp=str(today.tm_year-1)+"-"+str(today.tm_mon-1)+"-30"
    ticker_yahoo = yf.download(lista_tickers,start=emp)
    data = ticker_yahoo.Close.iloc[-2]
    return list(data)
last_prices=last_price(symbols)
last_prices[:20]

[*********************100%***********************]  506 of 506 completed


[2818.8701171875,
 429.95001220703125,
 1010.0,
 265.0,
 3105.56005859375,
 4.550000190734863,
 2733.969970703125,
 59.599998474121094,
 2291.7900390625,
 2094.0,
 138.36000061035156,
 180.0,
 1272.81005859375,
 12.300000190734863,
 7410.0,
 3119.659912109375,
 1735.43994140625,
 2024.199951171875,
 191.4600067138672,
 497.30999755859375]

In [96]:
df_last_prices=pd.DataFrame([(com,round(lp,2)) for com, lp in zip(names,last_prices)])
df_last_prices.columns=['Compañia','Precio']
df_last_prices


,Compañia,Precio
0,3D Systems Corporation,2818.87
1,3M Company,429.95
2,AGNC Investment Corp,1010.00
3,"ALPEK, S.A.B. de C.V",265.00
4,AMC ENTERTAINMENT HOLDINGS INC,3105.56
...,...,...
501,"Zoom Video Communications, Inc",111.00
502,"bluebird bio, Inc",980.69
503,eBay Inc,2210.46
504,"eXp World Holdings, Inc",1547.00
